In [ ]:
import pandas as pd

df = pd.read_csv(r"products.csv")
# Keep only rows with Price > 0
df = df.drop_duplicates().reset_index(drop=True)

df.to_csv("filterNewAkakce.csv",index=False)
print(f"Original rows: {len(df)}")

In [ ]:
import pandas as pd

# CSV dosyasını oku
df = pd.read_csv(r"C:\Users\pc\Desktop\AIbazaar\AIBazaar\AI\utils\filterPhone.csv")
print("İlk 10 satır:")
print(df.head(10))
print("\n" + "=" * 50 + "\n")

# Birleştirilecek kolonlar
columns_to_concat = [
    "storage",
    "ram",
    "phone_brand",
    "phone_model",  
    "dimensions",
    "display_size",
    "display_resolution",
    "os",
    "battery",
    "video",
    "chipset",
    "cpu",
    "gpu",
    "ppi_density",
]

# Yeni DataFrame oluştur
result_df = pd.DataFrame(columns=["productID", "Product Name", "description"])

# Her satır için işlem yap
for index, row in df.iterrows():
    # Boş olmayan değerleri key:value formatında birleştir
    description_parts = []

    for col in columns_to_concat:
        if col in df.columns:  # Kolon varsa
            value = row[col]
            # NaN, None veya boş değilse ekle
            if pd.notna(value) and str(value).strip() != "":
                description_parts.append(f"{col}:{value}")

    # Tüm parçaları birleştir
    description = ";".join(description_parts)

    # Phone model değerini al
    phone_model = (
        row["phone_model"]
        if "phone_model" in df.columns and pd.notna(row["phone_model"])
        else ""
    )

    # Yeni DataFrame'e ekle
    result_df.loc[index] = [
        index + 1,
        phone_model,
        description,
    ]  # Corrected column order

    # İlk 5 sonucu göster (test için)

# CSV olarak kaydet
output_path = (
    r"C:\Users\pc\Desktop\AIbazaar\AIBazaar\AI\utils\products_descriptions.csv"
)
result_df.to_csv(output_path, index=False, encoding="utf-8")

print(f"\nSonuç CSV dosyası kaydedildi: {output_path}")
print("\nOluşturulan CSV yapısı:")
print(result_df.head())

In [ ]:
import pandas as pd

input_filename = r'C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\filterNewAkakce.csv'

df = pd.read_csv(input_filename)

df['Product Name'] = df['Product Name'].str.strip()

df.to_csv(input_filename, index=False)


In [ ]:
import pandas as pd

# CSV dosyasını oku
input_filename = r'C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\filterNewAkakce.csv'
df = pd.read_csv(input_filename)

# "Fiyatı" kolonu hariç diğer tüm kolon adlarını al
columns_except_fiyat = [col for col in df.columns if col != 'Price']

# Bu kolonlara göre tekrar eden satırları kaldır
unique_df = df.drop_duplicates(subset=columns_except_fiyat)

# Yeni CSV dosyasını kaydet
output_filename = r'C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\withoutDuplicatePrice.csv'
unique_df.to_csv(output_filename, index=False)

print(f"Fiyatı hariç tekrarsız veri kaydedildi: {output_filename}")


In [ ]:
import pandas as pd

# CSV dosyasını oku
input_filename = r"C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\notebooks\LstmPriceHistory.csv"
df = pd.read_csv(input_filename)

# İndeksi 1'den başlat
df.index = range(1, len(df) + 1)

# CSV'ye yaz
df.to_csv(input_filename, index=True)

print(len(df))


In [ ]:
import pandas as pd


df = pd.read_csv("epeyProductListid.csv")
print("before",len(df))
df = df.drop_duplicates(subset="ProductID")
print("after",len(df))
df.to_csv("epeyProductListid.csv", index=False)


df = pd.read_csv("products.csv")
print("before", len(df))
df =df.drop_duplicates()
print("after", len(df))
df.to_csv("products.csv")

In [ ]:
import pandas as pd
import re

# CSV'yi oku
df = pd.read_csv(r"C:\Users\EXCALIBUR\Desktop\projects\Okul Ödevler\AIBazaar\AI\utils\notebooks\LSTMPriceHistory.csv")

df["ProductName"] = df["ProductName"].apply(lambda x: re.sub(r'[\/:*?"<>|]', ' ', x))

# Sonuçları kontrol et
print(df["ProductName"].head())
df.to_csv("LSTMPriceHistory.csv",index=False)


In [ ]:
import pandas as pd

import pandas as pd
import re

# CSV dosyasını oku
df = pd.read_csv(
    r"C:\Users\pc\Desktop\AIbazaar\AIBazaar\AI\utils\notebooks\LSTMProduct1.csv"
)


# Model, RAM ve Hafıza'yı ayıklayan yardımcı fonksiyon
def extract_product_name(description):
    try:
      
        ram = re.search(r"RAM Kapasitesi:\s*([\d]+ GB)", description)
        storage = re.search(r"Dahili Hafıza:\s*([\d]+ GB)", description)

        ram_str = ram.group(1).strip() if ram else ""
        storage_str = storage.group(1).strip() if storage else ""

        return f"{ram_str} RAM {storage_str}".strip()
    except:
        return ""

df["Price"] = df["Price"].astype(str).str.split(".").str[0]  # Fiyatı tam sayıya çevir
# Yeni kolonu oluştur
df["Model"] = df["Model"]+" "+df["Description"].apply(extract_product_name)

df.to_csv("product_specs_en1.csv", index=False)

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler


class LSTMModelEvaluator:
    def __init__(self, data_path, model_path, scaler_path, look_back=20):
        self.data_path = data_path
        self.model_path = model_path
        self.scaler_path = scaler_path
        self.look_back = look_back

    def load_data(self):
        df = pd.read_csv(self.data_path)
        df["Price"] = pd.to_numeric(df["Price"], errors="coerce")
        df["RecordDate"] = pd.to_datetime(df["RecordDate"], errors="coerce")
        df = df.dropna(subset=["RecordDate", "Price"])
        return df

    def prepare_sequences(self, prices_scaled):
        x, y = [], []
        for i in range(len(prices_scaled) - self.look_back):
            x.append(prices_scaled[i : i + self.look_back])
            y.append(prices_scaled[i + self.look_back])
        return np.array(x), np.array(y)

    def evaluate_model_metrics(self, product_id):
        # Dosyalar kontrol ediliyor
        if not os.path.exists(self.model_path) or not os.path.exists(self.scaler_path):
            raise FileNotFoundError("Model veya scaler dosyası bulunamadı.")

        # Model ve scaler yükleniyor
        model = load_model(self.model_path)
        with open(self.scaler_path, "rb") as f:
            scaler = pickle.load(f)

        # Veri yükleniyor
        df = self.load_data()
        product_df = df[df["ProductID"] == int(product_id)].sort_values("RecordDate")

        if len(product_df) < self.look_back + 1:
            raise ValueError("Değerlendirme için yeterli veri yok.")

        prices = product_df["Price"].values.reshape(-1, 1)
        prices_scaled = scaler.transform(prices)
        x, y_scaled = self.prepare_sequences(prices_scaled)

        # Tahmin yapılıyor
        y_pred_scaled = model.predict(x, verbose=0)
        y_true = scaler.inverse_transform(y_scaled)
        y_pred = scaler.inverse_transform(y_pred_scaled)

        # MAE hesapla
        mae = mean_absolute_error(y_true, y_pred)

        # MAPE ve accuracy hesapla
        epsilon = 1e-8  # sıfıra bölme hatası için
        mape = np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100
        accuracy = 100 - mape

        return {
            "productId": int(product_id),
            "productName": product_df["ProductName"].iloc[0],
            "mae": round(mae, 2),
            "mape": round(mape, 2),
            "accuracy_percent": round(accuracy, 2),
            "sample_count": len(y_true),
        }


DATA_PATH = (
    r"C:\Users\pc\Desktop\AIbazaar\AIBazaar\AI\utils\notebooks\LSTMPriceHistory.csv"
)
MODEL_PATH = (
    r"C:\Users\pc\Desktop\AIbazaar\AIBazaar\AI\utils\models\general_lstm_model.h5"
)
SCALER_PATH = (
    r"C:\Users\pc\Desktop\AIbazaar\AIBazaar\AI\utils\models\general_scaler.pkl"
)

evaluator = LSTMModelEvaluator(DATA_PATH, MODEL_PATH, SCALER_PATH)
result = evaluator.evaluate_model_metrics(product_id=854787)

print(result)

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

# Tarih ve fiyat verisi (örnek)
raw_data = pd.read_csv("LSTMPriceHistory.csv", sep="\t", header=None, names=["RecordDate", "Price"])
# Veri işleme
lines = [line.strip() for line in raw_data.strip().split("\n")]
records = []
for line in lines:
    try:
        date_str, price_str = line.split("\t")
        date = datetime.strptime(date_str.strip(), "%d.%m.%Y")
        price = float(price_str.strip().replace(",", "."))
        records.append((date, price))
    except:
        pass

df = pd.DataFrame(records, columns=["RecordDate", "Price"])

# Eksik veriler yerine sabit fiyatlar ekle (örnekleme amacıyla)
while len(df) < 100:
    last_date = df.iloc[-1]["RecordDate"]
    next_date = last_date + pd.Timedelta(days=1)
    df = pd.concat([df, pd.DataFrame({"RecordDate": [next_date], "Price": [df["Price"].iloc[-1] + np.random.uniform(0, 5)]})], ignore_index=True)

# LSTM için veriyi hazırlama
scaler = MinMaxScaler()
scaled_prices = scaler.fit_transform(df[["Price"]])
sequence_length = 10

X, y = [], []
for i in range(len(scaled_prices) - sequence_length):
    X.append(scaled_prices[i:i+sequence_length])
    y.append(scaled_prices[i+sequence_length])
X, y = np.array(X), np.array(y)
X = X.reshape((X.shape[0], X.shape[1], 1))

# LSTM model tanımı
model = Sequential()
model.add(LSTM(50, activation="relu", input_shape=(sequence_length, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=30, verbose=1)

# 15 gün ileriye tahmin
forecast = []
last_sequence = scaled_prices[-sequence_length:]
for _ in range(15):
    input_seq = last_sequence.reshape((1, sequence_length, 1))
    pred_scaled = model.predict(input_seq, verbose=0)[0][0]
    forecast.append(pred_scaled)
    last_sequence = np.append(last_sequence[1:], [[pred_scaled]], axis=0)

# Skaler ters dönüşüm (normalize → gerçek fiyat)
forecast_prices = scaler.inverse_transform(np.array(forecast).reshape(-1, 1)).flatten()

# Tahmini yazdır
for i, price in enumerate(forecast_prices, 1):
    print(f"Gün {i}: {price:.2f} TL")

# Grafik çiz
plt.plot(range(1, 16), forecast_prices, marker='o')
plt.title("15 Günlük LSTM Fiyat Tahmini")
plt.xlabel("Gün")
plt.ylabel("Tahmini Fiyat (TL)")
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd

# Türkçe key -> İngilizce key sözlüğü
tr_to_en = {
    "Model": "Model",
    "Ekran Boyutu": "Display Size",
    "Ekran Teknolojisi": "Display Technology",
    "Piksel Yoğunluğu": "Pixel Density",
    "Batarya Kapasitesi": "Battery Capacity",
    "Kamera Çözünürlüğü": "Camera Resolution",
    "CPU Üretim Teknolojisi": "CPU Manufacturing",
    "İşletim Sistemi": "Operating System",
    "RAM Kapasitesi": "RAM",
    "Dahili Hafıza": "Internal Storage",
    "Hızlı Şarj Desteği": "Fast Charging Support",
    "Ekran Yenileme Hızı": "Screen Refresh Rate",
    "5G": "5G",
    "Price": "Price"
}

def translate_description(desc, tr_to_en):
    if pd.isna(desc):
        return desc
    parts = desc.split(";")
    new_parts = []
    for part in parts:
        if ":" in part:
            key, val = part.split(":", 1)
            key = key.strip()
            val = val.strip()
            new_key = tr_to_en.get(key, key)  # Çeviri yoksa orijinal key
            new_parts.append(f"{new_key}: {val}")
        else:
            new_parts.append(part.strip())
    return "; ".join(new_parts)

# CSV dosyasını oku
df = pd.read_csv("orijinal.csv")

# Description kolonundaki Türkçe keyleri İngilizce'ye çevir
df["Description"] = df["Description"].apply(lambda x: translate_description(x, tr_to_en))

# Yeni CSV dosyasını kaydet
df.to_csv("yeni_english_description.csv", index=False)


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# --- Veriyi hazırla (senin kodun) ---
dates = [datetime.today() - timedelta(days=i) for i in range(100)]
dates = sorted(dates)

np.random.seed(42)
usd_try = np.round(np.random.normal(loc=32, scale=0.3, size=100), 2)
usd_try = np.clip(usd_try, 31.5, 32.5)

usd_price = 900
fluctuation = np.random.normal(0, 100, size=100)
prices = usd_try * usd_price + fluctuation
prices = np.round(prices, 2)

df = pd.DataFrame({"date": dates, "usd_try": usd_try, "price_tl": prices})
df = df.sort_values("date").reset_index(drop=True)

# --- Sadece price_tl kullanacağız ---
data = df[["price_tl"]].values

# --- Ölçekleme ---
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)


# --- Sliding window oluştur ---
def create_dataset(dataset, time_step=10):
    X, Y = [], []
    for i in range(len(dataset) - time_step):
        X.append(dataset[i : (i + time_step), 0])
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)


time_step = 10
X, y = create_dataset(scaled_data, time_step)

# LSTM giriş şekli (num_samples, time_step, num_features)
X = X.reshape(X.shape[0], X.shape[1], 1)

# --- Eğitim ve test ayır ---
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# --- Modeli oluştur ---
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(optimizer="adam", loss="mean_squared_error")

# --- Modeli eğit ---
model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=8,
    verbose=1,
)

# --- Tahmin ---
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# --- Tahminleri orijinal skala döndür ---
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
y_train_orig = scaler.inverse_transform(y_train.reshape(-1, 1))
y_test_orig = scaler.inverse_transform(y_test.reshape(-1, 1))

# --- Grafik için ---
train_plot = np.empty_like(scaled_data)
train_plot[:, :] = np.nan
train_plot[time_step : len(train_predict) + time_step, 0] = train_predict[:, 0]

test_plot = np.empty_like(scaled_data)
test_plot[:, :] = np.nan
test_plot[len(train_predict) + (time_step * 2) : len(scaled_data), 0] = test_predict[
    :, 0
]

plt.figure(figsize=(12, 6))
plt.plot(df["date"], data, label="Gerçek Fiyat (TL)")
plt.plot(df["date"], train_plot, label="Train Tahmin")
plt.plot(df["date"], test_plot, label="Test Tahmin")
plt.xlabel("Tarih")
plt.ylabel("Fiyat (TL)")
plt.title("iPhone 13 TL Fiyatı - LSTM Tahmini")
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject